In [1]:
import networkx as nx
import numpy as np
from dimod import DiscreteQuadraticModel
from dwave.system import LeapHybridDQMSampler
from dwave.cloud import Client

In [89]:
# Modularization with DQM solver

def modularization(G):

    B=nx.modularity_matrix(G)
    num_partitions=int(np.sqrt(len(G.nodes())))
    
    partitions = range(num_partitions)
    # Initialize the DQM object
    dqm = DiscreteQuadraticModel()
    
    
    for i in G.nodes():
        dqm.add_variable(num_partitions, label=i)

        
   
    for i in G.nodes():
        for j in G.nodes():
            if i==j:
                continue #the algorithm skips the linear term in QUBO/Ising formulation as in k-community a node has to belong to one community, therefore there is no effect in the maximising constellation
            dqm.set_quadratic(i,j, {(c, c): ((-1)*B[i,j]) for c in partitions})
    
    

    # Initialize the DQM solver
    sampler = LeapHybridDQMSampler()
   # sampler = greedy.SteepestDescentSampler()
    #sampleset = sampler.sample(dqm)

    # Solve the problem using the DQM solver
    sampleset = sampler.sample_dqm(dqm)

    # get the first solution
    sample = sampleset.first.sample
    energy = sampleset.first.energy
    
    run_time=(sampleset.info['run_time'])*0.001 #total runtime in milliseconds

    # Count the nodes in each partition
    counts = np.zeros(num_partitions)
    
    #create communities as parameter for evaluation function
    communities=[]
    for k in partitions:
        comm=[]
        for i in sample:
            if sample[i]==k:
                comm.append(i)
        communities.append(set(comm))

    #compute number of nodes in each community
    for i in sample:
        counts[sample[i]] += 1
    
    return (communities, run_time, energy, counts,sample)